In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import matplotlib
import os
import cv2
import torch
import torch.nn as nn
import torch.optim as optim
import os
import config
import seaborn as sn
from datetime import datetime
import numpy as np
import torchvision
from multi_scale_unet import UNET
import cv2
import matplotlib.pyplot as plt
import torchvision.transforms as tvtransforms
import time
import torch
import segmentation_models_pytorch as smp
import metrics as smpmetrics
import albumentations as A
from tqdm import tqdm
from meter import AverageValueMeter
from albumentations.pytorch import ToTensorV2
from torch.utils.data import Dataset as BaseDataset
from PIL import Image
import torch.nn.functional as F
from torch.utils.data import Dataset
import albumentations as A
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import StepLR

import torchmetrics
from torchmetrics import ConfusionMatrix
from matplotlib import cm
from matplotlib.colors import ListedColormap
import matplotlib.patches as mpatches

import neptune.new as neptune
from neptune.new.types import File

from PIL import Image
import numpy as np
import os
import config

In [2]:
run = neptune.init(
    project="jmt1423/coastal-segmentation",
    source_files = ['./*.ipynb', './*.py'],
    api_token=config.NEPTUNE_API_TOKEN,
)  # your credentials

https://app.neptune.ai/jmt1423/coastal-segmentation/e/COAS-25
Remember to stop your run once you’ve finished logging your metadata (https://docs.neptune.ai/api-reference/run#.stop). It will be stopped automatically only when the notebook kernel/interactive console is terminated.


In [ ]:
# model parameters
ENCODER = "resnet18"
ENCODER_WEIGHTS = 'imagenet'
ACTIVATION = 'sigmoid'
BATCH_SIZE = 1
MODEL_NAME = 'unet'
LEARNING_RATE = 0.00001
loss = smp.losses.DiceLoss(mode='binary')
LOSS_STR = 'Dice Loss'
EPOCHS = 150
DEVICE = 'cuda'

# optimizer parameters
BETA1 = 0.9
BETA2 = 0.999
EPSILON = 1e-8
OPTIM_NAME = 'AdamW'

# image parameters
MIN_HEIGHT = 32
MIN_WIDTH = 512
TRAIN_MIN_HEIGHT = 32
TRAIN_MIN_WIDTH = 512

binary_result_paths = [
    'binary_results/0.png',
    'binary_results/1.png',
    'binary_results/2.png',
    'binary_results/3.png',
    'binary_results/4.png',
    'binary_results/5.png',
    'binary_results/6.png',
    'binary_results/7.png',
    'binary_results/8.png',
    'binary_results/9.png',
    'binary_results/10.png',
    'binary_results/11.png',
    'binary_results/12.png',
    'binary_results/13.png',
    'binary_results/14.png',
    'binary_results/15.png',
    'binary_results/pred_0.png',
    'binary_results/pred_1.png',
    'binary_results/pred_2.png',
    'binary_results/pred_3.png',
    'binary_results/pred_4.png',
    'binary_results/pred_5.png',
    'binary_results/pred_6.png',
    'binary_results/pred_7.png',
    'binary_results/pred_8.png',
    'binary_results/pred_9.png',
    'binary_results/pred_10.png',
    'binary_results/pred_11.png',
    'binary_results/pred_12.png',
    'binary_results/pred_13.png',
    'binary_results/pred_14.png',
    'binary_results/pred_15.png',
]

TRAIN_IMG_DIR = "../../CoastSat/data/blackpool/images/binary_train/"
TRAIN_MASK_DIR = "../../CoastSat/data/blackpool/images/binary_trainannot/"
TEST_IMG_DIR = "../../CoastSat/data/blackpool/images/binary_test/"
TEST_MASK_DIR = "../../CoastSat/data/blackpool/images/binary_testannot/"
# VAL_IMG_DIR = '../../CoastSat/data/blackpool/images/val/'
# VAL_MASK_DIR = '../../CoastSat/data/blackpool/images/valannot/'




In [ ]:
# log parameters to neptune
run['parameters/model/model_name'].log(MODEL_NAME)
run['parameters/model/encoder'].log(ENCODER)
run['parameters/model/encoder_weights'].log(ENCODER_WEIGHTS)
run['parameters/model/activation'].log(ACTIVATION)
run['parameters/model/batch_size'].log(BATCH_SIZE)
run['parameters/model/learning_rate'].log(LEARNING_RATE)
run['parameters/model/loss'].log(LOSS_STR)
run['parameters/model/device'].log(DEVICE)
run['parameters/optimizer/optimizer_name'].log(OPTIM_NAME)
run['parameters/optimizer/beta1'].log(BETA1)
run['parameters/optimizer/beta2'].log(BETA2)
run['parameters/optimizer/epsilon'].log(EPSILON)

In [ ]:
class Dataset(Dataset):
    """This method creates the dataset from given directories"""
    def __init__(self, image_dir, mask_dir, transform=None):
        """initialize directories
        :image_dir: TODO
        :mask_dir: TODO
        :transform: TODO
        """
        self._image_dir = image_dir
        self._mask_dir = mask_dir
        self._transform = transform
        self.images = os.listdir(image_dir)

    def __len__(self):
        """returns length of images
        :returns: TODO
        """
        return len(self.images)

    def __getitem__(self, index):
        """TODO: Docstring for __getitem__.
        :returns: TODO
        """
        img_path = os.path.join(self._image_dir, self.images[index])
        mask_path = os.path.join(self._mask_dir, self.images[index])
        image = np.array(Image.open(img_path).convert("RGB"))
        mask = np.array(Image.open(mask_path).convert("L"), dtype=np.float32)
        mask[mask == 255.0] = 1

        if self._transform is not None:
            augmentations = self._transform(image=image, mask=mask)
            image = augmentations["image"]
            mask = augmentations["mask"]

        return image, mask

In [ ]:
def get_loaders(
    train_dir,
    train_mask_dir,
    val_dir,
    val_mask_dir,
    batch_size,
    train_transform,
    val_transform,
    num_workers=4,
    pin_memory=True,
):
    """
    This method creates the dataloader objects for the training loops

    :train_dir: directory of training images
    :train_mask_dir: directory of training masks
    :val_dir: validation image directory

    :returns: training and validation dataloaders
    recall
    """
    train_ds = Dataset(train_dir,
        train_mask_dir,
        transform=train_transform
    )
    
    train_loader = DataLoader(
        train_ds,
        batch_size=batch_size,
        num_workers=num_workers,
        pin_memory=pin_memory,
        shuffle=True,
    )

    val_ds = Dataset(
        image_dir=val_dir,
        mask_dir=val_mask_dir,
        transform=val_transform,
    )

    val_loader = DataLoader(
        val_ds,
        batch_size=batch_size,
        num_workers=num_workers,
        pin_memory=pin_memory,
        shuffle=False,
    )

    return train_loader, val_loader

In [ ]:
train_transform = A.Compose([
    A.Resize(height=TRAIN_MIN_HEIGHT, width=TRAIN_MIN_WIDTH),
    A.Rotate(limit=35, p=1.0),
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.1),
    A.ShiftScaleRotate(shift_limit=0.2, scale_limit=0.2, rotate_limit=30, p=0.5),
    A.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2, always_apply=False, p=0.5),
    A.Downscale(scale_min=0.25, scale_max=0.25, interpolation=0, always_apply=False, p=0.5),
    A.Emboss(alpha=(0.2, 0.5), strength=(0.2, 0.7), always_apply=False, p=0.5),
    A.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225],
    ),
    ToTensorV2(),
], )

test_transform = A.Compose([
        A.Resize(MIN_HEIGHT, MIN_WIDTH),
        ToTensorV2(),
    ], )

In [ ]:
# get the dataloaders
trainDL, testDL = get_loaders(TRAIN_IMG_DIR, TRAIN_MASK_DIR,
                                           TEST_IMG_DIR, TEST_MASK_DIR,
                                           BATCH_SIZE, train_transform,
                                           test_transform)

In [ ]:
# initialize model
model = smp.Unet(
   encoder_name=ENCODER, 
   encoder_weights=ENCODER_WEIGHTS, 
   in_channels=3,
   classes=1,
   activation=ACTIVATION,
)

# wavelet_model = UNET(in_channels=3, out_channels=6).to(DEVICE)

# metrics have been defined in the custom training loop as giving them in a list object did not work for me


# define optimizer and learning rate
optimizer = optim.AdamW(params=model.parameters(), lr=LEARNING_RATE, betas=(BETA1, BETA2), eps=EPSILON)

In [ ]:
metrics = [
    smp.utils.metrics.IoU(threshold=0.5),
    smp.utils.metrics.Precision(threshold=0.5),
    smp.utils.metrics.Recall(threshold=0.5),
    smp.utils.metrics.Fscore(threshold=0.5),
]

def check_accuracy(metrics, loader, model, device='cpu'):
    """ Custom method to calculate accuracy of testing data
    :loader: dataloader objects
    :model: model to test
    :device: cpu or gpu
    """

    model.eval() # set model for evaluation
    metrics_meters = {metric.__name__: AverageValueMeter() for metric in metrics}
    with torch.no_grad():  # do not calculate gradients
        for x, y in loader:
            x = x.to(device).float()
            y = y.to(device).unsqueeze(1)
            #x = x.permute(0,1,3,2)
            preds = torch.sigmoid(model(x))
            
            for metric_fn in metrics:
                    metric_value = metric_fn(preds, y).cpu().detach().numpy()
                    metrics_meters[metric_fn.__name__].add(metric_value)
            
            metrics_logs = {k: v.mean for k, v in metrics_meters.items()}
            
    print(metrics_logs)
    #print([type(k) for k in metrics_logs.values()])
    
    # log metrics into neptune
    run['metrics/train/iou_score'].log(metrics_logs['iou_score'])
    run['metrics/train/f1_score'].log(metrics_logs['fscore'])
    run['metrics/train/precision'].log(metrics_logs['precision'])
    run['metrics/train/recall'].log(metrics_logs['recall'])
    
    model.train()

def save_predictions_as_imgs(loader,
                             model,
                             folder="binary_results/",
                             device='cpu',
                             ):
    """TODO: Docstring for save_predictions_as_imgs.
    :loader: TODO
    :model: TODO
    :folder: TODO
    :device: TODO
    :returns: TODO
    """
    model.eval()

    for idx, (x, y) in enumerate(loader):
        x = x.to(device=device).float()
        with torch.no_grad():
            preds = torch.sigmoid(model(x))
            preds = (preds > 0.5).float()
        torchvision.utils.save_image(preds, f"{folder}/pred_{idx}.png")
        torchvision.utils.save_image(y.unsqueeze(1), f"{folder}{idx}.png")
    
    model.train()

In [ ]:
scaler = torch.cuda.amp.GradScaler()

def train_wavelet(loader, model, optimizer, loss_fn, scaler):
    """TODO: Docstring for train_fn.

    :loader: TODO
    :model: TODO
    :optimizer: TODO
    :loss_fn: TODO
    :scaler: TODO
    :returns: TODO

    """
    loop = tqdm(loader)

    for batch_idx, (data, targets) in enumerate(loop):
        data = data.to(device=DEVICE).float()
        targets = targets.to(device=DEVICE)
        targets = targets.long()
        data = data.permute(0,3,1,2)  # correct shape for image
        targets = targets.squeeze(1)

        # forward
        with torch.cuda.amp.autocast():
            predictions = model(data.contiguous())
            loss = loss_fn(predictions, targets)

        # backward
        optimizer.zero_grad()
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        # update loop
        loop.set_postfix(loss=loss.item())
scheduler = StepLR(optimizer=optimizer, step_size=5, gamma=0.1)

# def train_fn(loader, model, optimizer, loss_fn, scaler):
#     """ Custom training loop for models

#     :loader: dataloader object
#     :model: model to train
#     :optimizer: training optimizer
#     :loss_fn: loss function
#     :scaler: scaler object
#     :returns:

#     """
#     loop = tqdm(loader)  # just a nice library to keep track of loops
#     model = model.to(DEVICE)# ===========================================================================
#     for batch_idx, (data, targets) in enumerate(loop):  # iterate through dataset
#         data = data.to(device=DEVICE).float()
#         targets = targets.to(device=DEVICE).float()
#         targets = targets.unsqueeze(1)
#         data = data.permute(0,3,2,1)  # correct shape for image# ===========================================================================
#         targets = targets.to(torch.int64)

#         # forward
#         with torch.cuda.amp.autocast():
#             predictions = model(data)
#             loss = loss_fn(predictions, targets)

#         # backward
#         optimizer.zero_grad()
#         scaler.scale(loss).backward()
#         scaler.step(optimizer)
#         scaler.update()
#         # loss_values.append(loss.item())
#         #run['training/batch/loss'].log(loss)

#         #update loop
#         loop.set_postfix(loss=loss.item())
def train_fn(loader, model, optimizer, loss_fn, scaler):
    """TODO: Docstring for train_fn.
    :loader: TODO
    :model: TODO
    :optimizer: TODO
    :loss_fn: TODO
    :scaler: TODO
    :returns: TODO
    """
    loop = tqdm(loader)
    model = model.to(DEVICE)
    
    for batch_idx, (data, targets) in enumerate(loop):
        data = data.to(device=DEVICE).float()
        targets = targets.unsqueeze(1).to(device=DEVICE).float()
        #data = data.permute(0,1,3,2)

        # forward
        with torch.cuda.amp.autocast():
            predictions = model(data)
            loss = loss_fn(predictions, targets)

        # backward
        optimizer.zero_grad()
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        

        run['metrics/train/loss'].log(loss.item())
        # update loop
        loop.set_postfix(loss=loss.item())

for epoch in range(EPOCHS):  # run training and accuracy functions and save model
    run['parameters/epochs'].log(epoch)
    train_fn(trainDL, model, optimizer, loss, scaler)
    #train_wavelet(trainDL, wavelet_model, optimizer, loss, scaler)
    check_accuracy(metrics, testDL, model, DEVICE)
    save_predictions_as_imgs(
        testDL,
        model,
        folder='./binary_results/',
        device=DEVICE,
    )
    torch.save(model, './binary_{}.pth'.format(MODEL_NAME))
    scheduler.step()
    
    # if epoch in {25, 50 ,75, 100, 125, 150, 175, 200, 225, 250, 275, 300, 325, 350, 375, 400}:
    #     run["model_checkpoints/{}".format(MODEL_NAME)].upload("./binary_{}.pth".format(MODEL_NAME))

    # if epoch in {100, 200, 300, 400}:
    #     for image_path in binary_result_paths:
    #             run["train/results"].log(File(image_path))

print("Files and directories in a specified path:")
xcd = 0
for filename in os.listdir(TEST_MASK_DIR):
    f = os.path.join(TEST_MASK_DIR,filename)
    if os.path.isfile(f):
        im_gray = cv2.imread(f, cv2.IMREAD_GRAYSCALE)
        (thresh, im_bw) = cv2.threshold(im_gray, 128, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
        cv2.imwrite(f, im_bw)
        xcd+=1